# sigmoid 해석적 의미
    선형결합에 의해 나온 결과 값이 2x+3일 때 x가 1이라면 sigmoid(2x+3) = sigmoid(5)
     -> e^5 와 e^0의 상대적인 비율을 구함 -> e^5 / e^5 + e^0 = 약 94프로 -> 확률값을 결과값으로 반
     일반화: e^x / e^x + e^0 -> 분모 분자를 e^x로 나누면 1 / 1+e^(-x)
     
     sigmoid의 문제점:
  1. 입력값이 양수 또는 음수로 갈수록 sigmoid 결과값의 기울기가 0으로 가서 나중에 back propagation에서 param을 갱신할 때 기울기 소실 문제가 생긴다
  2. 결과값이 0~1이기 때문에 각 은닉층의 반환값은 무조건 양수이고 이는 back propagation을 했을 때 모든 파라미터 w에 대해 미분값이 모두 같은 부호를 같게 되고-> 같은 방향으로 update되는데 학습을 zigzag형태로 만들어 느리게 만든다
  3. exponential의 computational  비용 문제가 있다.

# softmax의 해석적의미
    output 노드가 세개이고 각 노드들의 선형결합이 각각 2x+3, -x+5, -2x+1 이고 x=1이면 반환값은 각각 5, 4, -1이 된다. softmax는 각 클래스의 전체에서의 상대적인 비율을 나타내므로 첫번째 노드의 output은 e^5 / (e^5 + e^4 + e^-1) 이 된다. 주어진 ground truth 가 0,1,0 일 때 y 햇과 y의 차이인 loss를 줄이기 위한 방향으로 gradient descent가 진행된다.

    argmax를 쓰면 상대적인 비율을 따지지 않고 가장 큰값에 1을 걸고 나머지에 0을 걸어서 0 1 0 이런 식으로(output 노드가 3개일 때) 나온다. 근데 이때는 back propagation을 할 때 0이 존재하므로 계산이 안되는데 softmax를 쓰면 반환값이 5,4,-1 일때 1/5+4-1 이렇게 하기 보다 더 극단적으로 우세한 class를 가려내기 위해 exponential을 걸어서 e^5 / (e^5 + e^4 + e^-1)를 반환하고, 이렇게 하면 0이 존재하지 않으므로 back propagation도 가능해진다.

# tanh의 해석적의미
     하이퍼볼릭 탄젠트 함수는 시그모이드에서 반환값을 -1에서 1로 늘린 것이다. 이렇게 하면 다음 노드에 전달하는 입력값을 음수,양수 다양하게 주어서 sigmoid의 단점을 커버하는 장점이 있지만, 여전히 기울기 소실 문제가 남아있다.


# ReLU의 해석적의미
     input이 0이하면 0을 반환 0이상이면 입력값 그대로를 반환하여 기울기가 무한대까지 갈수있으므로 기울기 소실문제가 발생하지 않지만, 입력값이 음수일때는 기울기가 0이므로 back propagation에서 가중치가 업데이트 되는 노드가 있고(기울기값이 input값이므로 고대로 곱해주면 됨) 사라져버리는 노드(기울기가 0이므로 곱해도 0-> 뒤의 가중치를 갱신할 수 없음)가 있다.


# Batch Normalization
    batch 학습-> 전체 데이터셋을 mini batch로 쪼개서 (5개분반으로 쪼갠다면) 첫번쨰 학습은 1분반, 두번째 학습은 2분반 ... , 다섯번쨰 학습은 5분반 -> 전체 데이터셋 한번 학습 완료. 이렇게 쪼개서 학습
     sigmoid나 tanh를 보면 input값이 특정범위를 넘어서면 기울기가 0으로 죽어버린다. 따라서 활성화 함수의 입력값을 제한하기 위해서 batch normalization을 한다.
     입력값과 가중치의 선형결합으로 각 노드의 결과가 -15, 5, 0.1, 2, -3 ... 이렇게 다양하게 나왔을 때 이것을 활성화 함수의 입력값으로 넣기 전에 이것들을 가우시안 분포로 normalization한다 (x-x평균 / 표준편차) -> 각 값들은 평균이 0이고 표준편차가 1인 분포의 데이터로 변환이되고 이것들을 활성화함수에 넣으면 기울기가 0이 될 가능성이 훨씬 줄어들게 된다.
         근데 이렇게 선형결합으로 구한 값을 근본없이 normalization하게 되면 기울기를 살리려는 의도에는 부합하지만 원래 원하는 예측의 수행은 못할 수 있음. 데이터 중에서 몇개만 target에 부합하는데 가우시안 정규화를 통해 대다수의 데이터들이 target에 부합하게 될 수도 있음. 따라서 가우시안 정규화로 구한 배치 데이터에 대해 다시 파라미터인 감마, 베타 연산의 term을 추가해(이것도 학습으로 결정) 몇개는 원래의 데이터로 복원하는 장치를 마련한다.
 batch normalization으로 학습을 전부 진행하고 test data에 대해서 검증시에 batch-normalization이 stable하게 적용되게 하기 위해, training때 사용했던 normalization 값들 (mean, var)들을 평균을 내서 적용

# Optimization Method
    gradient descent의 변종들
     SGD는 현재의 기울기만 고려를 했다면 모멘텀은 현재의 기울기뿐만 아니라 과거의 행적까지 보는 것 -> 과거의 기울기까지 모두 고려를 하면 sgd에서는 지그재그로 움직였던 원인인 한번은 오르고 한번은 내리는 것이 모멘텀에서는 모두 합산돼서 어느정도 상쇄될 것이고 한방향으로 쭉 가던 것이라면 합산돼서 점점 크게 가게 될 것이므로 sgd보다 빠르게 이동한다
     이렇게 관성을 이용하면 local minima 문제를 해결할 수 있다.
     Adagrad는 각 dimension의 기울기의 차이가 너무 커서 생기는 지그재그의 문제를 해결하기 위해 나왔는데, 지금까지의 기울기의 제곱의 합을 루트를 취해서(캐시) 가중치를 갱신할 때 나눠준다. 학습이 쌓일수록 나누는 값이 커져서 기울기가 0에 다가갈수록 학습이 너무 느려지는 문제가 생기고 이를 방지하기 위해 RmsProp이 있다.
     RmsProp 은 decay rate을 곱해서 최근에 가중치의 변화가 미미하게 일어났다면 캐시를 forget하여 캐시를 0에 가깝게 하여 기울기 update를 일어나게 함
     Adam:  모멘텀 + RmsProp -> 과거의 기울기에는 가중을 0.9로하고 현재의 기울기는 가중을 0.1로하여 과거의 기울기에 더 중점을 두고 기울기를 설정하고 거기에서 RmsProp 처럼 루트 v로 나누어 학습률에 곱해 가중치를 갱신한다.

# Learning Rate
    기울기가 0으로 갈수록 학습률을 동일하게 한다면 0으로 수렴하는 값으로 가지 않고 주변만 왔다갔다 할 수 있음. 따라서 기울기가 0근처로 가게되면 더 fine tuning을 통해 수렴할 수 있게 해줘야함
     그 때 학습률을 더 줄여서 기울기를 조정하는 함수를 적용하는 것이 바람직. 또는 loss가 더 줄어들지 않는다면 그때 학습률을 좀더 줄이고 또 loss가 줄어들지 않으면 그때 학습률을 더 줄여보는 방식도 많이 씀
     학습률을 조정할 때는 0.1 - 0.3 - 0.9 이렇게 등비수열(이때는 r=3)로 늘리거나, 줄일 떄도 마찬가지로 줄임

# Regularization term
    Machine Learning_regularization 에 정리 해놓음. 
    elastic net은 L1과 L2를 합친 것. overfitting을 방지하는 다른 장치가 많아서 안 쓰는 경우도 많다고 함.


# Dropout
     매 학습마다 은닉층들의 일부 뉴런들이 동작하는 것을 생략하면 뉴런의 조합만큼 지수함수적으로 다양한 모델을 학습시키는 것과 같다. 학습데이터에 대해서는 정확도가 올라가지 않지만
     overfitting을 개선하여 test data에 대해서는 정확도를 올릴 수 있다. 구현 방법은 random으로 생성한 shape(model(wx+b))으로 사전에 정해놓은 p보다 작을 경우 살리고 아니면 0을 부여하는 행렬을 만들고
     그 행렬을 노드들에 곱해주면 임의로 노드를 dropout 시킬 수 있다. test할 때는 모든 노드를 사용. 대신 출력값마다 학습 때 정한 p를 곱하여 학습과 test의 차이를 상쇄시킴
 
